In [1]:
import pandas as pd
import csv
import sys
import re
import scipy
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from time import time

csv.field_size_limit(sys.maxsize)

131072

In [2]:
df = pd.read_pickle('../data/instances.pkl')
labels = list(set(df['target'].values))

In [3]:
X = []
Y = []

print("Preparing lists...")
for index, row in df.iterrows():
    X.append(row["source_code"])
    Y.append(row["target"])

Preparing lists...


In [4]:
print("Extracting features...")
cv = CountVectorizer(binary=True)
cv.fit(X)
instances = cv.transform(X)

Extracting features...


In [5]:
X_train, X_test, y_train, y_test = train_test_split(instances, Y, train_size = 0.75, random_state=42)

/Users/erickmaziero/virtualenvs/refact_env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# Default parameters

In [ ]:
svm_classifier = svm.SVC(random_state=42, verbose=1)
svm_classifier.fit(X_train, y_train)

/Users/erickmaziero/virtualenvs/refact_env/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[LibSVM]

In [ ]:
print("============ EVALUATION on test set:")
print(accuracy_score(y_test, svm_classifier.predict(X_test)))

# Hyperparametrization

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

param_dist = {'C': scipy.stats.expon(scale=100),
              'gamma': scipy.stats.expon(scale=.1),
              'kernel': ['rbf', 'linear', 'poly'],
              'class_weight':['balanced', None]}

svm_classifier2 = svm.SVC(random_state=42)

n_iter_search = 20
random_search = RandomizedSearchCV(svm_classifier2,
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   cv=5,
                                   n_jobs=-1)

start = time()
print("Hyperparameter tuning...")
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
  " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)
print("============ EVALUATION on test set:")
print(accuracy_score(y_test, random_search.best_estimator_.predict(X_test)))